In [1]:
# Import Packages and setup
import os
from collections import OrderedDict
import logging
import time

import numpy as np

from astropy.table import Table as ApTable

import scarlet
import scarlet.constraints as sc

def deblend(images, filters, psfs, catalog, coords, bg_rms, sources, save_path, key):
    blend = scarlet.blend.Blend(sources, images, bg_rms=bg_rms)
    t1 = time.time()
    blend.fit(200)
    t2 = time.time()
    #print("blend {0} took {1}s for {2} iterations".format(idx, t2-t1, blend.it))
    seds = []
    morphs = []
    coords = []
    init_coords = []
    for m,src in enumerate(blend.sources):
        morphs.append(blend.get_model(m=m, combine=False, combine_source_components=True, use_sed=False)[0])
        seds.append(src.sed[0])
        coords.append(src.center)
        init_coords.append(src._init_center)
    result = ApTable([init_coords, coords, seds, morphs], names=("init_center", "coords", "sed", "morph"))
    result.meta["time"] = t2-t1
    result.meta["iter"] = blend.it
    filename = os.path.join(save_path, "blend_{0}_{1}.fits".format(idx, key))
    #print("saving results to:", filename)
    result.write(filename, overwrite=True)

In [4]:
data_path = "/Users/fred/lsst/code/scarlet_paper/real_matched"
save_path = "/Users/fred/temp/symmetry"
errors = OrderedDict()
t0 = time.time()
sigmas = [1, 0.9, 0.8, 0.7, 0.5, 0.3, 0.1, 0.05, 0.01, 0.001, 0.0001, 0]
#sigmas = [1,0.8,0]
keys = []
for idx in range(100):
    try:
        data = np.load(os.path.join(data_path, 'images_{0}.npz'.format(idx)))
        images = data["images"]
        filters = data["filters"]
        psfs = np.load(os.path.join(data_path, "psfs.npz"))["psfs"]
        catalog = ApTable.read(os.path.join(data_path, "true_catalog_{0}.fits".format(idx)))
        bg_rms = [20 for _img in images]
        coords =catalog['y','x']
        
        success = []
        for sigma in sigmas:
            if sigma > 0:
                # Soft direct symmetry
                constraint = (sc.DirectSymmetryConstraint(sigma=sigma)
                              & sc.PositivityConstraint()
                              & sc.DirectMonotonicityConstraint())
                sources = [scarlet.source.ExtendedSource((src['y'],src['x']),images,bg_rms,
                                                         constraints=constraint.copy()) for src in catalog]
                key = "soft-{0}".format(sigma)
                deblend(images, filters, psfs, catalog, coords, bg_rms, sources, save_path, key)
                if key not in keys:
                    keys.append(key)
                success.append(key)

                """
                # Smooth, convex symmetry
                sources = [
                    scarlet.source.ExtendedSource(
                        (src['y'],src['x']),
                        images,
                        bg_rms,
                        constraints=[
                            sc.SmoothSymmetryConstraint(sigma=sigma)
                            & sc.PositivityConstraint()
                            & sc.DirectMonotonicityConstraint()
                        ]) for src in catalog]
                key = "smooth-{0}".format(sigma)
                deblend(images, filters, psfs, catalog, coords, bg_rms, sources, save_path, key)
                if key not in keys:
                    keys.append(key)"""
                
            else:
                # Use no symmetry constraint
                constraint = sc.PositivityConstraint() & sc.DirectMonotonicityConstraint() 
                sources = [scarlet.source.ExtendedSource((src['y'],src['x']),images,bg_rms,
                                constraints=constraint.copy()) for src in catalog]
                key = "none"
                deblend(images, filters, psfs, catalog, coords, bg_rms, sources, save_path, key)
                if key not in keys:
                    keys.append(key)
                success.append(key)
                # Use prox_g symmetry
                
                constraint = sc.SymmetryConstraint() & sc.PositivityConstraint() & sc.DirectMonotonicityConstraint() 
                sources = [scarlet.source.ExtendedSource((src['y'],src['x']),images,bg_rms,
                                constraints=constraint.copy()) for src in catalog]
                key = "prox-g"
                deblend(images, filters, psfs, catalog, coords, bg_rms, sources, save_path, key)
                if key not in keys:
                    keys.append(key)
                success.append(key)
                
    except Exception as e:
        print("error in {0}, successful deblends: {1}".format(idx, success))
        errors[idx] = e.__repr__()

np.save(os.path.join(save_path, "keys.npy"), keys)
np.save(os.path.join(save_path, "bg_rms.npy"), bg_rms)
print(errors)

OrderedDict()


In [5]:
errors

OrderedDict()